In [1]:
import os
os.environ['HF_HOME'] = '/media/lobby/KINGSTON/.cache/huggingface'
os.environ['HF_HOME']

'/media/lobby/KINGSTON/.cache/huggingface'

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [3]:
# file_name = "Doc1-page1"
file_name = "Doc1-page1_part_1"
# file_name = "Doc1-page1-resized"
# file_name = "google"
# file_name = "japanese"

input_file_ext = "jpeg"
# input_file_ext = "jpg"
# input_file_ext = "png"

# input_path = r"a.txt"
input_path = os.path.join("input", f"{file_name}.{input_file_ext}")
# input_path = os.path.join("input", f"{file_name}.png")

# output_path = "/content/aa.txt"
output_path = os.path.join("output", f"{file_name}.txt")
input_path, output_path

('input/Doc1-page1_part_1.jpeg', 'output/Doc1-page1_part_1.txt')

In [4]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"
# model_name = ""


# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, 
    torch_dtype="auto", 
    device_map="auto",
    # attn_implementation="flash_attention_2"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained(model_name)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


Content examples:

```json
        "content": [
            {
                "type": "image",
                "image":f"/media/lobby/KINGSTON/GitHub-SecaraR/whisper/notebooks/image-2-text/Qwen2-VL/{input_path}",
            },
            {"type": "text", "text": "Describe this image."},
        ],
```

In [5]:

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image":f"/media/lobby/KINGSTON/GitHub-SecaraR/whisper/notebooks/image-2-text/Qwen2-VL/{input_path}",
            },
            {"type": "text", "text": "Output all the text from this image."},
        ],
    }
]

messages

[{'role': 'user',
  'content': [{'type': 'image',
    'image': '/media/lobby/KINGSTON/GitHub-SecaraR/whisper/notebooks/image-2-text/Qwen2-VL/input/Doc1-page1_part_1.jpeg'},
   {'type': 'text', 'text': 'Output all the text from this image.'}]}]

In [6]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
# inputs = inputs.to("cpu")
# inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# clear file content
open(output_path, 'w').close()

# add content to file
with open(output_path, 'w', encoding='utf-8') as f:
    f.writelines(output_text)

output_path

'output/Doc1-page1_part_1.txt'